![image](https://www.ewatercycle.org/img/logo.png)

# Case study 1: replace subbasin in PCRGlobWB2.0 with a MARRMoT model
This notebooks demonstrates how to use eWaterCycle to combine the output of two very different models in a single experiment. We run PCRGlobWB2.0 for the Rhine basin, but we replace the Moselle sub-basin with a MARRMoT model.

PCRGlobWB2.0 (Edwin 20XX) is a distributed model written in python and MARRMoT (Knoben 20XX) is a suite of conceptual models written in Matlab. To make the difference as large as possible, we choose the simplest model available within MARRMoT: the m01, a single leaky bucket.

## Import statements
We'll be using the following modules

In [ ]:
# TODO: clean up code to consistently use the same convention! This is now a mess of 
# double imports because this notebook is constructed from two example notebooks 
# written by different team members at different times in the project.

#OS related 
from os import environ, remove
from os.path import abspath
import os
from pathlib import Path
from glob import glob
from configparser import ConfigParser

#Time and time object related
import time
from datetime import datetime
from cftime import num2date

#calculations and plotting
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io as sio
import xarray as xr
from scipy import optimize
from tqdm import tqdm

#cartography (drawing maps)
from cartopy.io import shapereader
from cartopy import crs

#hydrological specific non eWaterCycle
import hydrostats.metrics as hm
import hydrostats.visual as hv

#eWaterCycle specific
from ewatercycle.observation.grdc import get_grdc_data
from grpc4bmi.bmi_client_docker import BmiClientDocker

## functions needed

The following functions are needed for this experiment.

TO DO: all of these need to become part of libraries, or at least be moved to a supporting .py file that is imported above

In [ ]:
# This was addepted from the notebook that demonstrates how to run MARRMoT
# From that version, the following changes were made:

# - changed to accept a filename instead of constructing it
# - changed to get passed variables instead of using globals

def write_marrmot_config(parameters, catchment, period, forcing_file_loc, 
            config_file_loc , model_name="m_01_collie1_1p_1s", 
            solver={
                "name": "createOdeApprox_IE",  # IE:Implicit Euler.
                "resnorm_tolerance": float(0.1),
                "resnorm_maxiter": float(6),
            }, store_ini=float(1500) ):
    """Write model configuration file.

    Adds the model parameters to forcing file for the given period
    and catchment including the spinup year and writes this information
    to a model configuration file.
    """
    # get the forcing that was created with ESMValTool
    #forcing_file = f"marrmot-m01_{forcing}_{catchment}_{PERIOD['spinup'].year}_{PERIOD['end'].year}.mat"
    forcing_data = sio.loadmat(forcing_file_loc, mat_dtype=True)

    # select forcing data
    forcing_data["time_end"][0][0:3] = [
        period["end"].year,
        period["end"].month,
        period["end"].day,
    ]

    # combine forcing and model parameters
    forcing_data.update(
        model_name=model_name,
        parameters=parameters,
        solver=solver,
        store_ini=store_ini,
    )
    
    sio.savemat(config_file_loc, forcing_data)

In [ ]:
# This function takes an BMI model object, extracts variable and stores it 
# as an xarray object. For this to work, the variable does need to have a propper
# setup grid. See the BMI documentation on grids.

def var_to_xarray(model, variable):
    # Get grid properties from model (x = latitude !!)
    # could be speedup, lots of bmi calls are done here that dont change between updates
    shape = model.get_grid_shape(model.get_var_grid(variable))
    lat = model.get_grid_x(model.get_var_grid(variable))
    lon = model.get_grid_y(model.get_var_grid(variable))
    time = num2date(model.get_current_time(), model.get_time_units())

    # Get model data for variable at current timestep
    data = model.get_value(variable)
    data = np.reshape(data, shape)

    # Create xarray object
    da = xr.DataArray(data, 
                      coords = {'longitude': lon, 'latitude': lat, 'time': time}, 
                      dims = ['latitude', 'longitude'],
                      name = variable,
                      attrs = {'units': model.get_var_units(variable)}
                     )

    # Masked invalid values on return array:
    return da.where(da != -999)

In [ ]:
# The following two functions translate lat,lon coordinates into BMI model 
# indices, which are used to get and set variable values.

def lat_lon_to_closest_variable_indices(model, variable, lats, lons):
    
    #get shape of model grid and lat-lon coordinates of grid
    shape = model.get_grid_shape(model.get_var_grid(variable))
    latModel = model.get_grid_x(model.get_var_grid(variable))
    lonModel = model.get_grid_y(model.get_var_grid(variable))
    nx = len(latModel)
    
    #for each coordinate given, determine where in the grid they fall and 
    #calculate 1D indeces
    if len(lats) == 1:
        idx = np.abs(latModel - lats).argmin()
        idy = np.abs(lonModel - lons).argmin()
        output = idx+nx*idy
    else:
        output=[]
        for [lat,lon] in [lats,lons]:
            idx = np.abs(latModel - lat).argmin()
            idy = np.abs(lonModel - lon).argmin()
            output.append(idx+nx*idy) 

    return np.array(output)

def lat_lon_boundingbox_to_variable_indices(model, variable, latMin, latMax, lonMin, lonMax):
    #get shape of model grid and lat-lon coordinates of grid
    shape = model.get_grid_shape(model.get_var_grid(variable))
    latModel = model.get_grid_x(model.get_var_grid(variable))
    lonModel = model.get_grid_y(model.get_var_grid(variable))
    nx = len(latModel)

    idx = [i for i,v in enumerate(latModel) if ((v > latMin) and (v < latMax))]
    idy = [i for i,v in enumerate(lonModel) if ((v > lonMin) and (v < lonMax))]
    
    output = []
    for x in idx:
        for y in idy:
            output.append(x + nx*y)
    
    return np.array(output)